<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/Python/Dynamic_Crawling_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 동적 크롤링2 - 네이버 쇼핑 구매하기

## 소개 및 로그인

### 라이브러리

1. 셀레니움
  - By
  - WebDriverWait
  - expected_conditions
  - Keys
  
2. pyperclip
  - 클립보드에 있는 내용을 복사


In [ ]:
import time
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

### 강의와 다른 변경점

- 셀레니움 4.x에서 webdriver를 패키지로 가져와서 자동 업데이트 가능

In [ ]:
# 드라이버 자동 업데이트
# pip install webdriver_manager
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

### 옵션 및 Wait 설정

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

wait = WebDriverWait(chrome, 10)
short_wait = WebDriverWait(chrome, 3)

### 로그인하기

- Explicity waits에 CSS_SELECTOR를 자주 사용하므로 `find`함수로 정의

- F12를 눌러 로그인 버튼에 해당하는 class를 찾아서 로그인 페이지로 이동

- ID와 PW에 해당하는 input을 찾음

In [ ]:
chrome.get("https://shopping.naver.com/home")

def find(wait,css_selector, selector = By.CSS_SELECTOR):
    return wait.until(EC.presence_of_element_located((selector, css_selector)))
# login_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a#gnb_login_button")))
login_button = find(wait,"a#gnb_login_button").click()

# input_id = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,"input#id")))
input_id = find(wait,"input#id")
# input_pw = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,"input#pw")))
input_pw = find(wait,"input#pw")

- send_keys를 이용한 로그인은 자동로그인 방지(리캡챠)가 뜸

In [ ]:
input_id.send_keys("naver-id")
input_pw.send_keys("naver-password")
input_pw.send_keys("\n") # 엔터로 로그인

- 클립보드를 이용한 방법은 자동로그인을 피할 수 있음
  - pyperclip 이용

- `input_id.send_keys(Keys.COMMAND, "v")`을 이용해서
클립보드에 복사 - 붙여넣기 식으로 아이디, 비밀번호 입력

In [ ]:
# pip3 install pyperclip

pyperclip.copy("naver-id")
input_id.send_keys(Keys.COMMAND, "v") # mac

pyperclip.copy("naver-password")
input_pw.send_keys(Keys.COMMAND, "v")
input_pw.send_keys("\n") # 엔터

- 2023.1.19 기준 자주사용하는 기기를 등록하겠냐는 페이지가 나타남
  - 등록 안함을 선택

In [ ]:
# 자주사용하는 기기 등록안함
dontsave = find(wait,"a#new\.dontsave").click()

## 상품 검색 및 출력

### 명령어 입력창 찾기 방법 1

- 상품 검색을 위해 키워드를 입력할 input type을 찾음

  ```html
  <input type="text"  class="_searchInput_search_text_fSuJ6" title="검색어   입력" value="">
  ```
  - class의 `class="_searchInput_search_text_fSuJ6"` text 뒤의 글자는 무작위로 변경할 수 있음
    - 앞의 내용만으로 class의 선택자를 가져옴

In [ ]:
# 상품 검색
search = find(wait, "input[class^=_searchInput_search_text_")
search.send_keys("아이폰 케이스")
time.sleep(1)
search.send_keys("\n")

### 명령어 입력창 찾기 방법 2
- XPATH를 이용하여 `title = "검색어 입력"`인 부분을 찾기

In [ ]:
search = find(wait, "//input[@title='검색어 입력']", By.XPATH)
search.send_keys("아이폰 케이스")
time.sleep(1)
search.send_keys("\n")

## 상품 검색 결과 및 무한 스크롤

### 상품 검색 결과

- "아이폰 케이스"를 검색했을 때 상품 목록을 가져옴
- 여러 항목을 가져오기 위해 `find_elements` 와 `for`문 사용


In [ ]:
# 상품 목록 노드 찾기
find(wait, "div[class^=basicList_info_area__")

In [ ]:
items = chrome.find_elements(By.CSS_SELECTOR, "a[class^=basicList_link__]")

- 강의 내용에서는 `find_elements_by_css_selector`였지만 셀레니움이 업데이트되면서 노드 검색하는 방법 변경
  - `browser.find_elements(By.CSS_SELECTOR, 'css')`

In [ ]:
# items = chrome.find_elements(By.CSS_SELECTOR, "div[class^=basicList_info_area__")
items = chrome.find_elements(By.CSS_SELECTOR, "a[class^=basicList_link__]")

# 상품 이름 가져와보기
for item in items:
    # 광고빼기
    # try:
    #     item.find_elements(By.CSS_SELECTOR,"button[class^=ad_]")
    #     continue
    # except:
    #     pass
    print(item.text)

- `find_elements`를 여러번 사용하였을 때 항목을 가져올 수 없었기 때문에 광고를 제외한 항목 가져오기 불가능  

  ![스크린샷 2023-01-18 오후 10 44 31](https://user-images.githubusercontent.com/96982072/213210819-ba7b299e-ca22-4b4d-9762-67c3fe77d07e.png)


- `item.text`를 이용한 결과 항목 다수 출력  
  - 항목이 5개 이하

  ![스크린샷 2023-01-18 오후 10 04 13](https://user-images.githubusercontent.com/96982072/213211053-f44a7a83-87e3-4d4d-8fa9-375bf1798200.png)


### 무한 스크롤

- 동적인 페이지에서 상품항목이 화면에 보이는만큼 로드함
  - 항목이 5개 이하
- 여러항목을 가져오기 위해 스크롤로 화면 밑으로 내려 검색결과를 최대한 로드
- 스크롤은 자바스크립트 명령어를 실행
  - `execute_script()`

In [ ]:
chrome.execute_script("window.scrollBy(0, document.body.scrollHeight)")

- 이 방법은 스크롤을 끝까지 내리지 못하고 코드를 종료
  - 반복문 사용

In [ ]:
for i in range(8):
    chrome.execute_script("window.scrollBy(0, document.body.scrollHeight)")
    time.sleep(1)

- 결과

  <img width="699" alt="스크린샷 2023-01-19 오전 12 28 53" src="https://user-images.githubusercontent.com/96982072/213213456-4ac95df8-b53a-42f6-840c-d5644bcdf4d9.png">

  - 상품 항목이 5개이상으로 늘어남